In [13]:
# This code is written by Yara Al-Shorman
# Date created: May 18 2022
# Last modified: May 22 2022
# Github repo: https://github.com/YaraAlShorman/Research-spring-22
# The purpose of this code is to predict the bulkflows for a large given set of galaxies (using machine learning)

### imports

In [34]:
# imports
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
# import matplotlib.pyplot as plt

### data preprocessing

In [22]:
# stacking all of the data into one humungous tensor
# I am using tensors because they can be ragged and can be stacked
# these means we won't have to cut off our data to standardize array sizes
data = np.load(f'C:/Users/yaras/Documents/Research/Feldman/rotated-outerrim-cz-rand/rotated-0-error-40.npy.npz')
input_data = tf.convert_to_tensor(data['data'])
scaler = MinMaxScaler()
scaler.fit(input_data)
input_data = scaler.transform(input_data)
input_data = tf.expand_dims(input_data, axis=0)
input_data = tf.RaggedTensor.from_tensor(input_data)

output_data = tf.convert_to_tensor(data['header'])
output_data = tf.expand_dims(output_data, axis=0)
# output_data = tf.RaggedTensor.from_tensor(output_data)

for i in range(1,10): # limit 3000
        array_data = np.load(f'C:/Users/yaras/Documents/Research/Feldman/rotated-outerrim-cz-rand/rotated-{i}-error-40.npy.npz')
        # 'data' data
        temp = tf.convert_to_tensor(array_data['data'], dtype='float64')
        scaler = MinMaxScaler()
        scaler.fit(temp)
        temp = scaler.transform(temp)
        temp = tf.expand_dims(temp, axis=0)  # dimensions have to be expandad to be able to concat along the outer-dimension 
        input_data = tf.concat([input_data, temp], axis=0)  # concat adds to existing dimensions, does not create new ones
        # 'header' data (bulkflows)
        temp = array_data['header']
        temp = np.expand_dims(temp, axis=0)
        output_data = np.concatenate([output_data, temp], axis=0)
        
# next is scaling

In [31]:
# normalizing/scaling the data
scaler = MinMaxScaler()
scaler.fit(output_data)
output_data = scaler.transform(output_data)
output_data = tf.convert_to_tensor(output_data)

# data is ready

In [39]:
def train_test_split_tensors(X, y, **options):
    """
    encapsulation for the sklearn.model_selection.train_test_split function
    in order to split tensors objects and return tensors as output

    :param X: tensorflow.Tensor object
    :param y: tensorflow.Tensor object
    :dict **options: typical sklearn options are available, such as test_size and train_size
    """

    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(np.asarray(X), np.asarray(y), **options)

    X_train, X_test = tf.constant(X_train), tf.constant(X_test)
    y_train, y_test = tf.constant(y_train), tf.constant(y_test)

    del(train_test_split)

    return X_train, X_test, y_train, y_test

In [40]:
# train-test splitting
train_test_split_tensors(input_data, output_data, train_size=0.85, random_state=0, shuffle=True)

TypeError: Singleton array array(<tf.RaggedTensor [[[0.04476167604186792, 0.4319348470437463, 0.6600869858736284,
   0.10562281276603319],
  [0.04470801316027669, 0.2964123450175338, 0.07774368327560849,
   0.45829554731734073],
  [0.03172237834438992, 0.3748306898537166, 0.5074990849844845,
   0.07828420800789393],
  ...,
  [0.9203440848389758, 0.9166299008698995, 0.9097912849111669,
   0.46985195993497325],
  [0.9973402144941484, 0.9373032164146093, 0.03445411172523899,
   0.13522825316059572],
  [1.0, 0.9376738124403152, 0.8614850196591373, 0.15025120160109273]],
 [[0.04476167604186792, 0.35549937554079314, 0.11415318614489606,
   0.2616085383021353],
  [0.03172237834438992, 0.35488049713897984, 0.11140956913496375,
   0.1683793761334571],
  [0.04253692822809806, 0.313280451454661, 0.6326202464701759,
   0.1588511671586666],
  ...,
  [0.9102376093501364, 0.9098281299488606, 0.741198125000201,
   0.30017373612523124],
  [0.9973402144941484, 0.9216550949622406, 0.27213734061030426,
  

In [32]:
print(input_data.shape[0], output_data.shape[0])

10 10


### model creation

In [ ]:
# model creations

model0 = tf.keras.models.Sequential(
    [
        # add layers here
        # figure out what layers to use
    ]
)
# add model stuff here